In [1]:
client_id = "86ac0958ab204b55bf2159768e863ea8"
client_secret = "e99322bd21d2460a94f3401976022220"

In [2]:
import spotipy
import requests
import matplotlib 
from spotipy.oauth2 import SpotifyOAuth

/Users/alexandriawashington/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:


scope = 'user-read-recently-played'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri = "https://github.com/wasnalex/SI206-Final-Project.git", scope=scope, show_dialog = True ))



In [4]:
#limit = number of tracks it will return, after = is a timestamp in miliseconds(basically songs played '9-5' but we give the time in unix), before = same but before. CANNOT USE BEFORE AND AFTER AT THE SAME TIME
recent = sp.current_user_recently_played(limit=5, after=None, before=None)